In [2]:
# Imports --- All of this may not be vital

from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from pprint import pprint
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import *
from imblearn.pipeline import Pipeline
# Homemade functions required
from data_prep_functions import *


In [3]:
### import data used to train classifiers ###

plasma_total_data_names = pd.read_excel("data/"+'gt15_plasma_features_names_biopy.xlsx', header=0, index_col=0)
csf_total_data_names = pd.read_excel("data/"+'gt15_csf_features_names_biopy.xlsx', header=0,index_col=0)

## sort into names and features
features_plasma = plasma_total_data_names.copy()
features_plasma = features_plasma.drop(['Corona'], axis=1)
names_plasma = plasma_total_data_names['Corona'].copy()

features_csf = csf_total_data_names.copy()
features_csf = features_csf.drop(['Corona'], axis=1) 
names_csf = csf_total_data_names['Corona'].copy()

### create a merged set
features_plasma_labeled = features_plasma.copy()
features_csf_labeled = features_csf.copy()

features_plasma_labeled['phase_plasma'] = 1
features_csf_labeled['phase_plasma'] = 0

features_merged = features_plasma_labeled.append(features_csf_labeled, ignore_index=True)
names_merged = names_plasma.append(names_csf, ignore_index=True)

# set with no phase labeling names are identical to names merged
features_merged_naive = features_merged.drop(['phase_plasma'], axis=1)

# print(plasma_total_data_names.shape, csf_total_data_names.shape, features_test.shape) ## in case you need to see shapes

## there is a known error here, sometimes there is an Unnamed column just drop it code is available in a 
#lower cell (scaling cell), its a holdover from two merged sets

scaler = MinMaxScaler()
total_data = features_merged_naive.copy()  ## for a regular netsurfp included case
# total_data = subset_features.copy() ### for a subset case --- use this one
total_data = total_data.fillna(0)
total_data_with_names = total_data.copy()
total_data = total_data.drop(['Protein names'], axis=1)
scaler = scaler.fit(total_data)
scaled_df = pd.DataFrame(scaler.transform(total_data), columns=total_data.columns)
print(scaled_df.shape)



scaled_df_phase = scaled_df.copy()
scaled_df_phase['phase_plasma'] = features_merged['phase_plasma'].copy()

plasma_data = scaled_df_phase[scaled_df_phase.phase_plasma==1]
plasma_data = plasma_data.drop(['phase_plasma'], axis=1)
scaled_df_plasma = plasma_data #pd.DataFrame(scaler.transform(plasma_data), columns=plasma_data.columns)

csf_data = scaled_df_phase[scaled_df_phase.phase_plasma==0]
csf_data = csf_data.drop(['phase_plasma'], axis=1)
scaled_df_csf = csf_data #pd.DataFrame(scaler.transform(csf_data), columns=csf_data.columns)




# scaled_df = scaled_df.drop(['Unnamed: 0.1'], axis=1)
# scaled_test_df = scaled_test_df.drop(['Unnamed: 0.1'], axis=1)

(174, 89)


In [3]:
scaled_df.head()

,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,frac_aa_G,frac_aa_H,frac_aa_I,frac_aa_K,frac_aa_L,...,fraction_exposed_exposed_S,fraction_exposed_exposed_T,fraction_exposed_exposed_V,fraction_exposed_exposed_W,fraction_exposed_exposed_Y,nsp_secondary_structure_coil,nsp_secondary_structure_sheet,nsp_secondary_structure_helix,nsp_disordered,asa_sum_normalized
0,0.457240,0.561143,0.300342,0.471036,0.746791,0.010886,0.208986,0.186978,0.811229,0.522934,...,0.198797,0.399849,0.299104,0.072222,0.250980,0.163934,0.000000,0.836066,0.038,0.164590
1,0.268538,0.288588,0.318724,0.247340,0.180208,0.252494,0.254702,0.560934,0.701237,0.262321,...,0.202440,0.131174,0.545589,0.103610,0.810127,0.483607,0.716763,0.080796,0.074,0.201285
2,0.214455,0.273754,0.207921,0.268716,0.445427,0.081323,0.148683,0.118245,0.599058,0.247915,...,1.000000,0.589823,0.365223,0.000000,0.288288,0.277518,0.990366,0.120609,0.009,0.293308
3,0.048290,0.266288,0.153181,0.191396,0.339209,0.156703,0.216942,0.191700,0.689105,0.284388,...,0.671720,0.637246,0.187414,0.108174,0.375918,0.396956,0.899807,0.056206,0.012,0.238334
4,0.354733,0.559535,0.340318,0.239531,0.486404,0.244341,0.216528,0.290021,0.673440,0.414400,...,0.268621,0.256670,0.189878,0.000000,0.126233,0.384075,0.358382,0.398126,0.030,0.101516


In [16]:

# be careful this exponentially increases runtime
params = {'sm__k_neighbors':[5, 7, 10, 12, 15],
          'sm__sampling_strategy': ['all'],
          'rf__max_depth': [10, 15, None],
          'rf__max_leaf_nodes': [None], 
          'rf__n_estimators': [200, 500, 700, 1000],
          'rf__criterion': ['entropy', 'gini'],
          'rf__max_features': ['sqrt', 'log2'],
          'rf__min_impurity_decrease': [0.02],
          'rf__ccp_alpha': [0.01]

         }
         #           'rf__max_depth': [5, 10, 15, 20, None],
        #   'rf__max_features': ['sqrt', 'log2'],
                #   'rf__max_leaf_nodes': [2, 5, 10, None],                'rf__max_depth': [5, 10, 15, 20, None],
                #         'rf__min_impurity_decrease': [0.0, 0.05, 0.1, 0.2]       'rf__criterion': ['entropy', 'gini'],


sss = StratifiedShuffleSplit(n_splits=100, test_size=0.1, random_state=2016)
rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
smote = SMOTE(sampling_strategy=1, random_state=2016, n_jobs=-1)
clf_rf = RandomForestClassifier(n_estimators=100, random_state=2016)
    
pipeline = Pipeline([
        ('sm', smote),
        ('rf', clf_rf)])
    
rf_random = GridSearchCV(estimator = pipeline, param_grid = params, cv = sss, n_jobs = -1, scoring='accuracy', verbose=2) #alter scoring as required
X_new = SelectKBest(f_classif, k=40).fit_transform(scaled_df, names_merged)
rf_random.fit(X_new, names_merged)



Fitting 100 folds for each of 240 candidates, totalling 24000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:   39.7s
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 4877 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 5808 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 6821 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 7914 tasks      | elapsed: 19.2min
[Parallel(n_jobs=-1)]: Done 9089 tasks      | 

In [15]:
print(rf_random.cv_results_['mean_test_score'].max(), rf_random.cv_results_['mean_test_score'].min())
pprint(rf_random.best_params_)

0.6413029193029193 0.5452063900152134
{'rf__ccp_alpha': 0.01,
 'rf__criterion': 'entropy',
 'rf__max_depth': 10,
 'rf__max_features': 'sqrt',
 'rf__max_leaf_nodes': None,
 'rf__min_impurity_decrease': 0.02,
 'rf__n_estimators': 200,
 'sm__k_neighbors': 12,
 'sm__sampling_strategy': 'all'}
